In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from torchsummary import summary
import os

# Check if CUDA is available
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure a GPU is available and CUDA is properly installed.")

device = torch.device("cuda")

# Load dataset (CIFAR-10 as an example)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Define CNN model
class CNN(nn.Module):
    def __init__(self, kernel_size, pooling_type="max"):
        super(CNN, self).__init__()
        if pooling_type == "max":
            self.pool = nn.MaxPool2d(2, 2)
        elif pooling_type == "avg":
            self.pool = nn.AvgPool2d(2, 2)
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size // 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function
def train_model(model, optimizer, scheduler, epochs=10, early_stopping_patience=5):
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    
    train_losses = []
    test_accuracies = []
    best_accuracy = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / len(test_loader.dataset)
        test_accuracies.append(accuracy)

        scheduler.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Early Stopping
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    return train_losses, test_accuracies

# Hyperparameter configurations
kernel_sizes = [3, 5, 7]
pooling_types = ["max", "avg"]
epochs_options = [5, 50, 100, 250, 350]
optimizers = {"SGD": optim.SGD, "RMSProp": optim.RMSprop, "Adam": optim.Adam}
batch_sizes = [64, 128, 256, 512]

# Running experiments
results = {}
for batch_size in batch_sizes:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    for kernel_size in kernel_sizes:
        for pooling in pooling_types:
            for opt_name, opt_func in optimizers.items():
                for epochs in epochs_options:
                    print(f"\nTraining model with Batch Size: {batch_size}, Kernel Size: {kernel_size}, Pooling: {pooling}, Optimizer: {opt_name}, Epochs: {epochs}")
                    model = CNN(kernel_size=kernel_size, pooling_type=pooling)
                    optimizer = opt_func(model.parameters(), lr=0.001)
                    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

                    train_losses, test_accuracies = train_model(model, optimizer, scheduler, epochs=epochs, early_stopping_patience=10)
                    results[(batch_size, kernel_size, pooling, opt_name, epochs)] = (train_losses, test_accuracies)

# Save results and generate output files
os.makedirs("results", exist_ok=True)
for config, (train_losses, test_accuracies) in results.items():
    batch_size, kernel_size, pooling, opt_name, epochs = config
    with open(f"results/result_bs{batch_size}_ks{kernel_size}_{pooling}_{opt_name}_ep{epochs}.txt", "w") as f:
        f.write(f"Train Losses: {train_losses}\n")
        f.write(f"Test Accuracies: {test_accuracies}\n")

print("\nAll experiments completed. Results saved to 'results/' directory.")


Files already downloaded and verified
Files already downloaded and verified

Training model with Batch Size: 64, Kernel Size: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5, Loss: 2.2926, Accuracy: 18.79%
Epoch 2/5, Loss: 2.2641, Accuracy: 25.36%
Epoch 3/5, Loss: 2.2128, Accuracy: 26.85%
Epoch 4/5, Loss: 2.1271, Accuracy: 27.90%
Epoch 5/5, Loss: 2.0323, Accuracy: 31.62%

Training model with Batch Size: 64, Kernel Size: 3, Pooling: max, Optimizer: SGD, Epochs: 50
Epoch 1/50, Loss: 2.2894, Accuracy: 20.70%
Epoch 2/50, Loss: 2.2584, Accuracy: 23.35%
Epoch 3/50, Loss: 2.1995, Accuracy: 26.28%
Epoch 4/50, Loss: 2.1119, Accuracy: 28.68%
Epoch 5/50, Loss: 2.0234, Accuracy: 31.10%
Epoch 6/50, Loss: 1.9464, Accuracy: 34.13%
Epoch 7/50, Loss: 1.8829, Accuracy: 35.61%
Epoch 8/50, Loss: 1.8325, Accuracy: 37.03%
Epoch 9/50, Loss: 1.7902, Accuracy: 38.54%
Epoch 10/50, Loss: 1.7537, Accuracy: 39.71%
Epoch 11/50, Loss: 1.7325, Accuracy: 39.67%
Epoch 12/50, Loss: 1.7290, Accuracy: 39.75%
Epoch 13

KeyboardInterrupt: 